In [1]:
import pandas as pd
import cvxpy as cp
import numpy as np
import docplex.mp.model as md


# 加载数据


In [2]:

# phase = np.loadtxt("phase")
# cir=phase[0].sum()
# phase=phase/cir
# vol = np.loadtxt("volume")
# qv = np.loadtxt("path_vol")
# phase_get = np.load("phase_get.npy")
# d = np.loadtxt("distance")
# dr = np.loadtxt("dr")
# dr=dr/cir
# straight_get = np.load("straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])
# px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


g = np.array(
    [
        [87, 67, 71, 68, 79, 82, 69, 75, 80, 69, 75, 78, 68, 83, 78],
        [87, 67, 71, 68, 79, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 180, 80, 69, 75, 78, 68, 83, 78],
        [0, 37, 71, 72, 79, 82, 70, 75, 80, 70, 70, 78, 60, 83, 78],
        [0, 0, 0, 0, 0, 0, 0, 0, 180, 70, 70, 78, 60, 25, 0],
        [87, 67, 71, 68, 79, 82, 180, 0, 0, 0, 0, 0, 0, 0, 0],
    ]
)
g = g / 180

q = np.array([187, 347, 285, 102, 114, 403])
Q = np.array(
    [
        [534, 534, 534, 534, 534, 187, 187, 472, 472, 472, 472, 472, 472, 472, 472],
        [403, 403, 505, 505, 505, 505, 102, 102, 102, 214, 214, 214, 214, 102, 102],
    ]
)

dr = np.array(
    [
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 37, 0, 35, 0, 0, 36, 0, 0, 36, 35, 0, 35, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 36, 35, 0, 35, 32, 0],
        [0, 37, 0, 35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    ]
)
dr = dr / 180

d = [236, 457, 346, 227, 226, 236, 200, 135, 117, 402, 404, 403, 354, 234]


## 速度数据


In [3]:
lower, upper = 8, 12
# linspace = np.arange(lower, upper, 0.5)
# lin_num = len(linspace)

# # 南北直行方向绿灯时间
# sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
# sr=1-sg

## 长干道数据


In [4]:
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
# nump = len(phase_get)
M = 100000000
nx = 0.00000001
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

# g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])

r=1-g

# 定义变量


### 通用变量

In [5]:
# model = md.Model("prosmodel")
# Z_list = [(i) for i in range(num)]
# z = model.continuous_var_dict(Z_list, lb=1 / 180, ub=1 / 80, name="z")

# o_list = [(i) for i in range(num)]
# o = model.continuous_var_dict(o_list, lb=0, ub=1, name="o")

# t_list = [(i, k) for i in range(2) for k in range(num-1)]
# t = model.continuous_var_dict(t_list, lb=0, name="t")


### 速度波动变量

In [6]:
# yp_list=[(i,j,k) for i in range(2) for j in range(num) for k in range(lin_num)]
# yp=model.binary_var_dict(yp_list,name="yp")

# nt_list = [(i, j, k) for i in range(2) for j in range(num) for k in range(lin_num)]
# nt = model.integer_var_dict(nt_list, lb=-1, ub=10, name="nt")


### 长干道变量

In [7]:
model = md.Model("prosmodel")
Z_list = [(i) for i in range(num)]
z = model.continuous_var_dict(Z_list, lb=1 / 180, ub=1 / 80, name="z")

p_list = [(i) for i in range(num)]
p = model.binary_var_dict(p_list, name="p")

o_list = [(i) for i in range(num)]
o = model.continuous_var_dict(o_list, lb=0, ub=1, name="o")

t_list = [(i, k) for i in range(2) for k in range(num-1)]
t = model.continuous_var_dict(t_list, lb=0, name="t")

w_list = [(i, k) for i in range(numr) for k in range(num)]
w = model.continuous_var_dict(w_list, lb=0, ub=1, name="w")


b_list = [(i, k) for i in range(numr) for k in range(num)]
b = model.continuous_var_dict(b_list, lb=0, ub=1, name="b")

n_list = [(i, k) for i in range(numr) for k in range(num)]
n = model.integer_var_dict(n_list, lb=0, ub=10, name="n")

u_list = [(i, k) for i in range(numr) for k in range(num)]
u = model.continuous_var_dict(u_list, lb=0, ub=1, name="u")

y_list = [(i, k) for i in range(numr) for k in range(num)]
y = model.binary_var_dict(y_list, name="y")

# 建立约束

## 长干道约束

保障路径$i$的绿波带宽出现出现在交叉口$k$内：
$$
\begin{cases}
b_{i,k}/2+\Delta_k\leq w_{i,k}\leq 1-r_{i,k}-b_{i,k}/2\\
\bar{b}_{i,k}/2+\bar{\Delta}_k\leq \bar{w}_{i,k} \leq 1-r_{i,k}-\bar{b}_{i,k}/2
\end{cases}
$$


In [8]:
for k in range(num):
    for i in range(numr):
        model.add_constraint(b[i, k] / 2 <= w[i, k])
        model.add_constraint(w[i, k] <= 1 - r[i][k] - b[i, k] / 2)


速度变化区间为：
$$
\begin{cases}
\frac{d_k}{f_k}z\leq t_k \leq \frac{d_k}{e_k}z\\
\frac{\bar{d}_k}{\bar{f}_k}z \leq \bar{t}_k \leq \frac{\bar{d}_k}{\bar{e}_k}z\\
\frac{d_k}{h_k}z\leq \frac{d_k}{d_{k+1}}t_{k+1}-t_k \leq \frac{d_k}{g_k}z\\
\frac{\bar{d}_k}{\bar{h}_k}z \leq \frac{\bar{d}_k}{\bar{d}_{k+1}}\bar{t}_{k+1}-\bar{t}_k\leq \frac{\bar{d}_k}{\bar{g_k}}z
\end{cases}
$$

In [9]:
for k in range(num-1):
    model.add_constraint(d[k] / speedScope[1] * z[k] <= t[0, k])
    model.add_constraint(t[0, k] <= d[k] / speedScope[0] * z[k])

    model.add_constraint(d[k] / speedScope[1] * z[k] <= t[1, k])
    model.add_constraint(t[1, k] <= d[k] / speedScope[0] * z[k])

for k in range(num-2):
    model.add_constraint(d[k] / speedVar[0] * z[k] <= d[k] / d[k+1] * t[0, k + 1] - t[0, k])
    model.add_constraint(d[k]/d[k+1] * t[0, k + 1] - t[0, k] <= d[k] / speedVar[1] * z[k])

    model.add_constraint(d[k] / speedVar[0] * z[k] <= d[k]/d[k+1] * t[1, k + 1] - t[1, k])
    model.add_constraint(d[k]/d[k+1] * t[1, k + 1] - t[1, k] <= d[k] / speedVar[1] * z[k])

确保每个子区至少包含两个交叉口：
$$p_k+p_{k+1}<=1$$

In [10]:
for k in range(num-1):
    model.add_constraint(p[k] + p[k + 1] <= 1)

交叉口偏移量$\mu_{i,k}$，表示当交叉口$k$为干道分割点时，路径$i$在交叉口的偏移量，即当$p_k=1$时，$\tau_{i,k}>0$,否则，$\mu_{i,k}=0$,约束表达为：
$$
\epsilon p_k\leq \mu_{i,k}\leq p_k
$$

In [11]:
for k in range(num):
    for i in range(numr):
        model.add_constraints([nx * p[k] <= u[i, k], u[i, k] <= p[k]])

当路径$i$在交叉口$k$取得非零带宽，则应满足如下约束:
$$\begin{cases}
beZ_k-M (1-y_{i,k})\leq b_{i,k}\leq y_{i,k} \\
beZ_k-M (1-\bar{y}_{i,k})\leq \bar{b}_{i,k}\leq \bar{y}_{i,k}
\end{cases}
$$

In [12]:
for k in range(num):
    for i in range(numr):
        model.add_constraint(be * z[k] - M * (1 - y[i, k]) <= b[i, k])
        model.add_constraint(b[i, k] <= y[i, k])


相同分段子区内部的交叉口绿波带宽相同，约束表达为：
$$\begin{cases}
-M p_{k+1}\leq b_{i,k}-b_{i,k+1}\leq M p_{k+1}\\
-M p_{k}\leq \bar{b}_{i,k}-\bar{b}_{i,k+1}\leq M p_{k}
\end{cases}
$$

为避免路径产生无效带宽，引入二进制变量$y_{i,k}$,其定义如下：
$$
y_{i,k}=\begin{cases}
1,&\text{如果路径$i$在交叉口$k$能够获得有效非零带宽}\\
0,& \text{其他}
\end{cases}
$$

路径$i$在相同分组内取得相等带宽，进而约束相同分组内交叉口同时取得非零带宽，在不同分组可能无法取得带宽，其约束表达如下：

$$\begin{cases}
-M p_{k+1}\leq y_{i,k}-y_{i,k+1}\leq M p_{k+1}\\
-M p_{k}\leq \bar{y}_{i,k}-\bar{y}_{i,k+1}\leq M p_{k}
\end{cases}
$$

其含义为，当路径$i$在交叉口$k$能够取得非零带宽,则其带宽应大于基础带宽$be$

因引入$y_{i,k}$，当路径$i$在交叉口$k$无法取得有效带宽时，其循环整形约束应不再起作用，循环整形约束可修改为:

$$
\begin{cases}
\theta_k+r_{i,k}+\varphi_{i,k}+w_{i,k}+t_{k}+\mu_{k+1}\geq \theta_{k+1}+r_{i,k+1}+\varphi_{i,k+1}+w_{i,k+1}+n_{i,k+1}-M(1-y_{i,k})\\
\theta_k+r_{i,k}+\varphi_{i,k}+w_{i,k}+t_{k}+\mu_{k+1}\leq \theta_{k+1}+r_{i,k+1}+\varphi_{i,k+1}+w_{i,k+1}+n_{i,k+1}+M(1-y_{i,k})\\ 
\end{cases}
$$

$$
\begin{cases}
\theta_k+\bar{r}_{i,k}+\bar{\varphi}_{i,k}+\bar{w}_{i,k}+\bar{n}_{i,k}\geq\theta_{k+1}+\bar{r}_{i,k+1}+\bar{\varphi}_{i,k+1}+\bar{w}_{i,k+1}+\bar{t}_{k}+\bar{\mu}_{k}-M(1-y_{i,k})\\
\theta_k+\bar{r}_{i,k}+\bar{\varphi}_{i,k}+\bar{w}_{i,k}+\bar{n}_{i,k}\leq\theta_{k+1}+\bar{r}_{i,k+1}+\bar{\varphi}_{i,k+1}+\bar{w}_{i,k+1}+\bar{t}_{k}+\bar{\mu}_{k}+M(1-y_{i,k})
\end{cases}
$$

In [13]:
def add_split_on_cons(i):
    for k in range(num-1):
        model.add_constraint(o[k] + r[i, k] + dr[i, k] + w[i, k] + t[0, k] + u[i, k]>=\
            o[k + 1]+ r[i, k+1] + dr[i, k+1]+ w[i, k + 1]+ n[i, k + 1]- M * (1 - y[i, k]))
        model.add_constraint(o[k] + r[i, k] + dr[i, k] + w[i, k] + t[0, k] + u[i, k]<=\
            o[k + 1]+ r[i, k+1] + dr[i, k+1]+ w[i, k + 1]+ n[i, k + 1]+ M * (1 - y[i, k]))

        model.add_constraint(-M * p[k + 1] <= b[i, k] - b[i, k + 1])
        model.add_constraint(b[i, k] - b[i, k + 1] <= M * p[k + 1])

        model.add_constraint(-M * p[k + 1] <= y[i, k + 1] - y[i, k])
        model.add_constraint(y[i, k + 1] - y[i, k] <= M * p[k + 1])
def add_split_in_cons(i):
    for k in range(num-1):
        model.add_constraint(o[k] + r[i, k] + dr[i, k] + w[i, k] + n[i, k]>=\
             o[k + 1]+ r[i, k+1]+ dr[i, k+1]+ w[i, k + 1]+ t[1, k]+ u[i, k]- M * (1 - y[i, k]))
        model.add_constraint(o[k] + r[i, k] + dr[i, k] + w[i, k] + n[i, k]<=\
             o[k + 1]+ r[i, k+1]+ dr[i, k+1]+ w[i, k + 1]+ t[1, k]+ u[i, k]+ M * (1 - y[i, k]))

        model.add_constraint(-M * p[k] <= b[i, k] - b[i, k + 1])
        model.add_constraint(b[i, k] - b[i, k + 1] <= M * p[k])

        model.add_constraint(-M * p[k] <= y[i, k + 1] - y[i, k])
        model.add_constraint(y[i, k + 1] - y[i, k] <= M * p[k])

In [14]:
for i in range(numr):
    if ison[i]==0:
        add_split_on_cons(i)
    elif ison[i]==1:
        add_split_in_cons(i)

注意到某些情况下，模型可能更偏好某一方向带宽，从而将其他方向的带宽设置为0，为避免这种情况的发生，同时体现模型对上行（下行）方向的偏好，定义约束：

$$
(1-\rho_{k})\sum_i b_{i,k}\geq (1-\rho_{k})\rho_{k} \sum_i \bar{b}_{i,k}
$$

In [15]:
for k in range(num):
    model.add_constraint((1 - rho[k]) * model.sum([b[i, k]*(1-ison[i]) for i in range(numr)])>=\
         (1 - rho[k])* rho[k]* model.sum([b[i, k]*ison[i] for i in range(numr)]))

因干道分割可对长干道进行分组，组内实现干道协调，不同组之间互不影响，因此不同分组可采用不同的信号协调周期，其约束表达如下：

$$
\begin{cases}
Z_{min}\leq Z_k\leq Z_{max}\\
-Mp_{k+1}\leq Z_{k+1}-Z_{k}\leq MP_{k+1}
\end{cases}
$$

In [16]:
for k in range(num-1):
    model.add_constraint(-M * p[k + 1] <= z[k + 1] - z[k])
    model.add_constraint(z[k + 1] - z[k] <= M * p[k + 1])

通过该约束，可实现在不同分段子区之间，可采用不同的信号周期，避免过长的干道采用相同信号周期造成部分交叉口车辆延误过大

## 速度波动约束

插入图片

$i$表示路径，$j$表示相位，$k$表示交叉口
- $A_k$ ：表示上行带宽所处红灯结束（绿灯开始时间）点位

- $B_k$: 表示上行带宽所处绿灯结束（红灯开始时间）点位

- $A_{k}^{\prime}$:上行带宽与交叉口相位的左侧交点

- $B_{k}^{\prime}$:上行带宽与交叉口相位的右侧交点

- $\bar{A}_{k}$：表示下行带宽所处红灯结束（绿灯开始时间）点位

-  $\bar{B}_{k}$: 表示下行带宽所处绿灯结束（红灯开始时间）点位

- $\bar{A}_{k}^{\prime}$:下行带宽与交叉口相位的左侧交点

- $\bar{B}_{k}^{\prime}$:下行带宽与交叉口相位的右侧交点

$$\begin{cases}
A_{k,v}=\theta_k+rs_k\\
B_{k,v}=\theta_k+r_k+gs_k\\
A_{k,v}^{\prime}=max\{A_{k-1,v}^{\prime}+d_{k-1}/v*z_k-M*(1-p_k),A_{k,v}+\tau_k+n_{k,v}\}\\
B_{k,v}^{\prime}=min\{B_{k-1,v}^{\prime}+d_{k-1}/v*z_k+M*(1-p_k),B_{k,v}+n_{k,v}\}\\
\bar{A}_{k}=\theta_k+\bar{rs}_k+\Delta_k\\
\bar{B}_{k}=\theta_k+\bar{rs}_k+\bar{gs}_k+\Delta_k\\
\bar{A}_{k,v}^{\prime}=max\{\bar{A}_{k+1,v}^{\prime}+d_{k}/v*z_k-M*(1-p_k),\bar{A}_{k,v}+\bar{\tau}_k+\bar{n}_{k,v}\}\\
\bar{B}_{k,v}^{\prime}=min\{\bar{B}_{k+1,v}^{\prime}+d_{k}/v*z_k+M*(1-p_k),\bar{B}_{k,v}+\bar{n}_{k,v}\}
\end{cases}
$$


In [17]:
# def add_var_on_cons(A, B, o, r, g, t, n, k, end, yp, px, p, sum_on,rho):
#     if k>=end:
#         return sum_on
#     else:
#         A=model.max([A+t[k-1]-M*(1-p[k]),o[k]+r[k]+n[k]+px[k]])
#         B=model.min(B+t[k-1]+M*(1-p[k]), o[k]+r[k]+g[k]+n[k]+px[k])
#         model.add_constraints([-M*(1-yp[k]) <= B-A, B-A <= g[k]+M*(1-yp[k])])
#         model.add_constraints([be*z[k]-M*(1-yp[k]) <= B-A, B-A <= yp[k]+M*(1-yp[k])])
#         sum_on+=(B-A)*yp[k]
#         return add_var_on_cons(A, B, o, r, g, t, n, k+1, end, yp, px, p, sum_on,rho)

# def add_var_in_cons(A, B, o, r, g, t, n, k, end, yp, px, p, sum_in,rho):
#     if k<=end:
#         return sum_in
#     else:
#         A=model.max([A+t[k]-M*(1-p[k]), o[k]+r[k]+n[k]+px[k]])
#         B=model.min([B+t[k]+M*(1-p[k]), o[k]+r[k]+g[k]+n[k]])
#         model.add_constraints([-M*(1-yp[k]) <= B-A, B-A <= g[k]+M*(1-yp[k])])
#         model.add_constraints([be*z[k]-M*(1-yp[k]) <= B-A, B-A <= yp[k]+M*(1-yp[k])])
#         sum_in += (B-A)*yp[k]
#         return add_var_in_cons(A, B, o, r, g, t, n, k-1, end, yp, px, p, sum_in,rho)

In [18]:
# sum_on=0
# sum_in=0
# for i,v in enumerate(linspace):
#     sum_on+=add_var_on_cons(
#         A=o[0]+sr[0,0],
#         B=o[0]+sr[0,0]+sg[0,0],
#         o=o,
#         r=sr[0],
#         g=sg[0],
#         t=np.array([d[j]/v*z[j] for j in range(num-1)]),
#         n=np.array([nt[0,j,i] for j in range(num)]),
#         k=1,
#         end=num,
#         yp=np.array([yp[0,j,i] for j in range(num)]),
#         px=px[0],
#         p=p,
#         sum_on=sum_on,
#         rho=rho
#     )
#     sum_in+=add_var_in_cons(
#         A=o[num-1]+sr[1,num-1],
#         B=o[num-1]+sr[1,num-1]+sg[1,num-1],
#         o=o,
#         r=sr[1],
#         g=sg[1],
#         t=np.array([d[j]/v*z[j] for j in range(num-1)]),
#         n=np.array([nt[1,j,i] for j in range(num)]),
#         k=num-2,
#         end=-1,
#         yp=np.array([yp[1,j,i] for j in range(num)]),
#         px=px[1],
#         p=p,
#         sum_in=sum_in,
#         rho=rho       
#     )

为避免某些速度下无法取得有效带宽，进而影响其他速度下的带宽，可尽量
$$
\eta_{k,v}=\begin{cases}
1,&\text{如果速度$v$在交叉口$k$下能够获得有效带宽}\\
0,& \text{其他}
\end{cases}
$$
$$
\begin{cases}
be*z_k-M(1-\eta_{k,v})\leq B^{\prime}_{k,v} -A ^{\prime}_{k,v} \leq \eta_{k,v}+M*(1-\eta_{k,v})\\
be*z_k-M(1-\bar{\eta}_{k,v})\leq B^{\prime}_{k,v} -A ^{\prime}_{k,v} \leq \bar{\eta}_{k,v}+M*(1-\bar{\eta}_{k,v})
\end{cases}
$$

为了保证带宽式中为正，应确保点$B^{\prime}_k(\bar{B}^{\prime}_k)$始终处于点$A^{\prime}_k(\bar{A}^{\prime}_k)$的右侧，约束表示为：
$$
\begin{cases}
-M*(1-\eta_{k,v})\leq B^{\prime}_{k,v}-A^{\prime}_{k,v}\leq g_{k}+M*(1-\eta_{k,v})\\
-M*(1-\bar{\eta}_{k,v} )\leq\bar{B}^{\prime}_{k,v}-\bar{A}^{\prime}_{k,v} \leq \bar{g}_k+M*(1-\bar{\eta}_{k,v})
\end{cases}
$$

$$
\begin{cases}
b_{k,v}=B_{k,v}^{\prime}-A_{k,v}^{\prime},& k=1,2,...,n,&v \in \Omega_v \\
\bar{b}_{k,v}=\bar{B}_{k,v}^{\prime}-\bar{A}_{k,v}^{\prime},& k=0,1,...,n-1,&v \in \Omega_v
\end{cases}
$$

$$
(1-\rho_{k})\sum_i b_{k,v}\geq (1-\rho_{k})\rho_{k} \sum_i \bar{b}_{k,v}
$$

$$
\begin{cases}
-Mp_{k+1}\leq\eta_{k,v}-\eta_{k+1,v}\leq Mp_{k+1}\\
-Mp_{k}\leq\bar{\eta}_{k,v}-\bar{\eta}_{k+1,v}\leq Mp_{k}
\end{cases}
$$

In [19]:
# for k in range(num-1):
#     for v in range(lin_num):
#         model.add_constraints([-M*p[k+1]<=yp[0,k,v]-yp[0,k+1,v],yp[0,k,v]-yp[0,k+1,v]<=M*p[k+1]])
#         model.add_constraints([-M*p[k]<=yp[1,k,v]-yp[1,k+1,v],yp[1,k,v]-yp[1,k+1,v]<=M*p[k]])

# 构建目标函数

## 长干道目标构建

目标函数包括三部分：
- 最大化每条路径各交叉口的的绿波带宽。
- 最小化分割点停车等待时间。
- 最小化分割次数，并限制分割点流量。
$$
MAX:\sum_{i=1}^{I}\left(\omega_1\sum_{k=1}^{\Omega_i}\left(q_{i}b_{i,k}+\bar{q}_{i}\bar{b}_{i,k} \right)-\omega_2
\sum_{k=1}^{\Omega_i}\left(q_{i}\mu_{i,k}+\bar{q}_{i}\bar{\mu}_{i,k}\right)\right)-\omega_3\sum_{k=1}^{\Omega_i}\left(p_k\left(Q_k+\bar{Q}_k\right)\right)
$$

In [20]:
sum_b = model.sum([q[i] * b[i, k] for i in range(numr) for k in range(num)])
sum_u = model.sum([q[i] * u[i, k] for i in range(numr) for k in range(num)])
sum_p = model.sum([p[k] * (Q[0, k] + Q[1, k]) for k in range(num)])
sum_split=sum_b*5-sum_u*4-sum_p*1

## 速度波动目标构建

$$
MAX: \sum_{v\in\Omega}\left(\omega_1\sum_{k=1}^{I}\eta_{k,v}(B^{\prime}_{k,v}-A^{\prime}_{k,v})+\bar{\eta}_{k,v}(\bar{B}^{\prime}_{k,v}-\bar{A}^{\prime}_{k,v})\right)
$$

In [21]:
# sum_var=sum_in+sum_on

In [22]:
model.maximize(sum_split)
sol = model.solve()
print(sol.solve_details)
sol.objective_value

status  = integer optimal, tolerance
time    = 2.063 s.
problem = MILP
gap     = 0.00987066%



60005.003740460146

In [23]:
o = sol.get_value_dict(o)
# nt = sol.get_value_dict(nt)
z = sol.get_value_dict(z)
t = sol.get_value_dict(t)
p = sol.get_value_dict(p)
w = sol.get_value_dict(w)
n = sol.get_value_dict(n)
u = sol.get_value_dict(u)
y = sol.get_value_dict(y)
b = sol.get_value_dict(b)
# yp = sol.get_value_dict(yp)

In [24]:
Df=[[i for i in range(1,num+1)]]
Df+=[[d[i] for i in range(num-1)] + [np.nan]]
Df+=[[b[i,k] for k in range(num)] for i in range(numr)]
Df+=[[o[k] for k in range(num)]]
Df+=[[p[k] for k in range(num)]]
Df+=[[t[i,k] for k in range(num-1)]+[np.nan] for i in range(2)]
Df+=[[y[i,k] for k in range(num)] for i in range(numr)]
Df+=[[z[k] for k in range(num)]]
Df+=[[u[i,k] for k in range(num)] for i in range(numr)]
# Df+=[[yp[i,k,j] for k in range(num)] for j in range(lin_num) for i in range(2)]
Df=np.array(Df)
Df=Df.T
Df=pd.DataFrame(Df)
cols=["cross_number"]
cols+=["distance"]
cols+=["b_"+str(i) for i in range(numr)]
cols+=["offset","p"]
cols+=["t_"+str(i) for i in range(2)]
cols+=["y_"+str(i) for i in range(numr)]
cols+=["z"]
cols+=["u_"+str(i) for i in range(numr)]
# cols+=["yp_on_"+str(i) for i in range(lin_num)]
# cols+=["yp_in_"+str(i) for i in range(lin_num)]
Df.columns=cols

In [25]:
Df[["b_"+str(i) for i in range(numr)]+["y_"+str(i) for i in range(numr)]]

,b_0,b_1,b_2,b_3,b_4,b_5,y_0,y_1,y_2,y_3,y_4,y_5
0,0.269613,0.370833,0.000000,0.000000,0.000000,0.483333,1.0,1.0,0.0,0.0,0.0,1.0
1,0.269613,0.370833,0.000000,0.205556,0.000000,0.372222,1.0,1.0,0.0,1.0,0.0,1.0
2,0.244376,0.366581,0.000000,0.205556,0.000000,0.372222,1.0,1.0,0.0,1.0,0.0,1.0
3,0.244376,0.366581,0.000000,0.211111,0.000000,0.377778,1.0,1.0,0.0,1.0,0.0,1.0
4,0.308889,0.222222,0.000000,0.211111,0.000000,0.377778,1.0,1.0,0.0,1.0,0.0,1.0
5,0.308889,0.222222,0.000000,0.000000,0.000000,0.455556,1.0,1.0,0.0,0.0,0.0,1.0
6,0.235556,0.000000,0.000000,0.000000,0.000000,0.455556,1.0,0.0,0.0,0.0,0.0,1.0
7,0.235556,0.000000,0.000000,0.416667,0.000000,0.000000,1.0,0.0,0.0,1.0,0.0,0.0
8,0.383333,0.000000,0.383333,0.416667,0.000000,0.000000,1.0,0.0,1.0,1.0,0.0,0.0
9,0.383333,0.000000,0.383333,0.388889,0.388889,0.000000,1.0,0.0,1.0,1.0,1.0,0.0


In [26]:
Df[["t_"+str(i) for i in range(2)]]

,t_0,t_1
0,0.368750,0.368750
1,0.636364,0.594929
2,0.387457,0.387457
3,0.169466,0.169466
4,0.104630,0.104630
5,0.163889,0.163889
6,0.150000,0.150000
7,0.067500,0.067500
8,0.095937,0.095937
9,0.494444,0.494444
